In [1]:
# Configs
dataset_dir = r'datasets\PhishStorm\urlset.csv'

In [2]:
# import zlib
# file_name = '600_650_batch_ugxHSbe5cpWquoeKBxEcEPaz'
# with open(rf'TestsOnMaliciousURL1\ChatGPTData\{file_name}', 'rb') as f:
#     decompressed = zlib.decompress(f.read()).decode()

# with open(rf'TestsOnMaliciousURL1\ChatGPTData\{file_name}.jsonl', 'wt') as f:
#     f.write(decompressed)

In [3]:
import pandas as pd
import numpy as np
import torch
import json
from copy import copy, deepcopy
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import pandas as pd
from torch.utils.data import Dataset, DataLoader, TensorDataset
from transformers import BertTokenizer
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize, TweetTokenizer
import time
import matplotlib.pyplot as plt
import random
from torch_scatter import scatter_max
from torch_scatter import scatter_mean
from torch_scatter import scatter_sum
from torch_scatter import scatter_std
import torchmetrics
import lightning as L
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from torch.utils.flop_counter import FlopCounterMode

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 1024
device

device(type='cuda')

In [5]:
df = pd.read_csv(dataset_dir, on_bad_lines='warn', encoding = 'ISO-8859-1', low_memory=False)[['domain','label']]
df.columns=['url', 'Topic']
df.dropna(inplace=True)
class_list = df.Topic.unique()
class_id = {'benign':0, 'phishing': 1}
id_class = {0: 'benign', 1:'phishing'}

C:\Users\fardin\AppData\Local\Temp\ipykernel_3824\2769126135.py:1: ParserWarning: Skipping line 18259: expected 14 fields, saw 15
Skipping line 18273: expected 14 fields, saw 15

  df = pd.read_csv(dataset_dir, on_bad_lines='warn', encoding = 'ISO-8859-1', low_memory=False)[['domain','label']]


In [6]:
class_id

{'benign': 0, 'phishing': 1}

In [7]:
# url_id = 50000
# print(df.iloc[url_id].url)
# print(df.iloc[url_id].Topic)

In [8]:
# template_dict = {"custom_id": "", "method": "POST", "url": "/v1/embeddings", "body": {"model": "text-embedding-3-small", "input": "","max_tokens": 1000}}
# intervals = 50000
# previous = 0
# for i in range(intervals, len(df)+intervals, intervals):
#     i = min(len(df), i)
#     print(previous, i)
#     list_of_urls = []
#     for j in range(previous, i):
#         url = df['url'].iloc[j]
#         template_dict["custom_id"] = f"url-{j}"
#         template_dict["body"]["input"] = f"Please analyze this URL to determine if it is benign or a phishing attempt: {url}"
#         list_of_urls.append(json.dumps(template_dict))
        
#     with open(fr"TestsOnPhishStorm\ChatGPTData\Phishstorm_{previous}_{i}.jsonl", 'ta') as f:
#         for url in list_of_urls:
#             f.write(url + "\n")
#     previous = i
    

In [9]:
# import zlib
# file_key_values = {
#     'batch_DaM6p01fF0aA6BJcdd5t6kx2': '0_50_query_batch_DaM6p01fF0aA6BJcdd5t6kx2.jsonl',
#     'batch_z6Wz2WMfGnZATGNhwIhFgVJZ': '50_95_query_batch_z6Wz2WMfGnZATGNhwIhFgVJZ.jsonl',
#     'batch_pxLpTNJW2KZdzMOoaBYaNNft': '0_50_batch_pxLpTNJW2KZdzMOoaBYaNNft.jsonl',
#     'batch_lx9otx1fgDqjx2Y0YU75CYCO': '50_95_batch_lx9otx1fgDqjx2Y0YU75CYCO.jsonl',
# }

# for k, v in file_key_values.items():
#     with open(rf'TestsOnPhishStorm\ChatGPTData\{k}', 'rb') as f:
#         uncompressed = zlib.decompress(f.read()).decode()

#     with open(rf'TestsOnPhishStorm\ChatGPTData\{v}', 'wt') as f:
#         f.write(uncompressed)

In [74]:
GPTEmbeddingPaths = [
    r'TestsOnPhishStorm\ChatGPTData\0_50_batch_pxLpTNJW2KZdzMOoaBYaNNft.jsonl',
    r'TestsOnPhishStorm\ChatGPTData\50_95_batch_lx9otx1fgDqjx2Y0YU75CYCO.jsonl'
    # r'TestsOnPhishStorm\ChatGPTData\0_50_query_batch_DaM6p01fF0aA6BJcdd5t6kx2.jsonl',
    # r'TestsOnPhishStorm\ChatGPTData\50_95_query_batch_z6Wz2WMfGnZATGNhwIhFgVJZ.jsonl'
]

In [75]:
data_embeddings = {}
for emb_path in GPTEmbeddingPaths:
    with open(emb_path, 'rt') as f:
        all_lines = []
        for line in f.readlines():
            all_lines.append(json.loads(line))
        for url_data in all_lines:
            data_embeddings[int(url_data['custom_id'][4:])] = np.array(url_data['response']['body']['data'][0]['embedding'])

In [76]:
embedding_dim = len(url_data['response']['body']['data'][0]['embedding'])
embeddings = torch.zeros((len(data_embeddings), embedding_dim))
labels = torch.zeros((len(data_embeddings), len(class_id)))
for i, (k, v) in enumerate(data_embeddings.items()):
    embeddings[i] = torch.from_numpy(v)
    labels[i][int(df['Topic'].iloc[k])] = 1

In [77]:
train_embs, test_embs, train_labels, test_labels = train_test_split(embeddings, labels, test_size=0.1, shuffle=True)
train_dataset = TensorDataset(train_embs, train_labels)
test_dataset = TensorDataset(test_embs, test_labels)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, drop_last=True, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, drop_last=True, shuffle=False)

In [78]:
class GPTEmbeddingClassifier(torch.nn.Module):
    
    def __init__(self, input_dim, num_classes, *args, **kwargs) -> None:
        super(GPTEmbeddingClassifier, self).__init__(*args, **kwargs)
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, num_classes)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [79]:

class ClassifierLightningModel(L.LightningModule):
    def __init__(
        self,
        model,
        num_classes,
        optimizer=None,
        loss_func=None,
        learning_rate=0.01,
        batch_size=64,
        lr_scheduler=None,
        user_lr_scheduler=False,
        min_lr=0.0,
    ):
        super(ClassifierLightningModel, self).__init__()
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.model = model
        self.min_lr = min_lr
        # self.save_hyperparameters(ignore=["model"])
        self.save_hyperparameters("model", logger=False)
        self.optimizer = self._get_optimizer(optimizer)
        self.lr_scheduler = (
            self._get_lr_scheduler(lr_scheduler) if user_lr_scheduler else None
        )
        self.loss_func = loss_func
        self.train_losses = []
        self.val_losses = []
        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)
        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=num_classes)

    def forward(self, x, *args, **kwargs):
        return self.model(x)

    # def on_train_epoch_start(self) -> None:
    #     param_groups = next(iter(self.optimizer.param_groups))
    #     if "lr" in param_groups and param_groups["lr"] is not None:
    #         current_learning_rate = float(param_groups["lr"])
    #         self.log(
    #             "lr",
    #             current_learning_rate,
    #             batch_size=self.batch_size,
    #             on_epoch=True,
    #             on_step=False,
    #         )

    def training_step(self, batch, *args, **kwargs):
        X, y = batch
        X.to(self.device)
        y.to(self.device)
        
        self.model.train()
        y_out = self(X)

        loss = self.loss_func(y_out.view(y.shape), y )
        self.train_losses.append(loss.detach().item())
        self.log(
            "train_loss",
            loss,
            prog_bar=True,
            batch_size=self.batch_size,
            on_epoch=True,
            on_step=True,
        )
        
        self.train_acc(torch.argmax(y_out, dim=1), torch.argmax(y, dim=1))
        self.log('train_acc', self.train_acc, prog_bar=True, on_epoch=True, on_step=True, batch_size=self.batch_size)
        
        return loss

    def validation_step(self, batch, *args, **kwargs):
        X, y = batch
        X.to(self.device)
        y.to(self.device)
        
        self.model.eval()
        y_out = self(X)
        loss = self.loss_func(y_out.view(y.shape), y )
        self.val_losses.append(loss.detach().item())

        self.log(
            "val_loss",
            loss,
            prog_bar=True,
            batch_size=self.batch_size,
            on_epoch=True
        )
        
        self.val_acc(torch.argmax(y_out, dim=1), torch.argmax(y, dim=1))
        self.log('val_acc', self.val_acc, prog_bar=True, on_epoch=True, batch_size=self.batch_size)

    def configure_optimizers(self):
        if self.lr_scheduler is None:
            return self.optimizer

        return {
            "optimizer": self.optimizer,
            "lr_scheduler": {
                "scheduler": self.lr_scheduler,
                "monitor": "train_loss",
                "interval": "epoch",
                "frequency": 1,
            },
        }

    def update_learning_rate(self, learning_rate: float):
        self.learning_rate = learning_rate
        for g in self.optimizer.param_groups:
            g["lr"] = learning_rate

    def _get_optimizer(self, optimizer):
        return (
            optimizer
            if optimizer is not None
            else torch.optim.Adam(self.model.parameters(), lr=self.learning_rate)
        )

    def _get_lr_scheduler(self, lr_scheduler):
        return (
            lr_scheduler
            if lr_scheduler is not None
            else torch.optim.lr_scheduler.ReduceLROnPlateau(
                self.optimizer, patience=5, factor=0.5, mode="min", min_lr=self.min_lr
            )
        )


In [80]:
lr=0.00339
input_dim = 1536 
num_classes = len(class_id)

In [81]:
# for i in range(5):
callbacks = [
    # EarlyStopping(monitor='val_loss',mode='min',patience=25),
    # CustomModelCheckpoint(dirpath=r'models\malicious_urls_model', filename=f'malicious_urls_model_', every_n_epochs=3, mode='min', monitor='val_loss_epoch', save_on_train_epoch_end=True),
    ModelCheckpoint(save_top_k=5, mode='max', monitor='val_acc', save_last=True)
    ]

classifier_torch_model = GPTEmbeddingClassifier(embedding_dim, num_classes).to(device)
optimizer = torch.optim.Adam(classifier_torch_model.parameters(), lr=lr, weight_decay=0.00001)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[25, 50, 75, 100, 125, 150, 175],gamma=0.5)
loss_func = torch.nn.BCEWithLogitsLoss()
classfier_lightning_model = ClassifierLightningModel(classifier_torch_model, 
                                                    num_classes=num_classes,
                                            learning_rate=lr,
                                            batch_size=batch_size,
                                            optimizer=optimizer,
                                            loss_func=loss_func,
                                            lr_scheduler=lr_scheduler,
                                            user_lr_scheduler=False
                                            ).to(device)

from lightning.pytorch.loggers import CSVLogger
import lightning as L

trainer = L.Trainer(
            callbacks=callbacks,
            max_epochs=200,
            accelerator= 'gpu' if torch.cuda.is_available() else 'cpu',
            logger=CSVLogger(save_dir='logs/', name='log2')
        )

c:\Users\fardin\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\utilities\parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [82]:
# from lightning.pytorch.tuner import Tuner
# tuner = Tuner(trainer)
# result = tuner.lr_find(classfier_lightning_model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader, min_lr=0.000001, max_lr=0.1, num_training=2000)
# fig = result.plot(suggest=True)
# fig.show()

In [83]:
X, y = next(iter(test_dataloader))
flopt_counter = FlopCounterMode(classfier_lightning_model.model)
with flopt_counter:
    classfier_lightning_model.model(X.to(device))

Module                            FLOP    % Total
---------------------------  ---------  ---------
GPTEmbeddingClassifier       1880.097M    100.00%
 - aten.addmm                1880.097M    100.00%
 GPTEmbeddingClassifier.fc1  1610.613M     85.67%
  - aten.addmm               1610.613M     85.67%
 GPTEmbeddingClassifier.fc2   268.435M     14.28%
  - aten.addmm                268.435M     14.28%
 GPTEmbeddingClassifier.fc3     1.049M      0.06%
  - aten.addmm                  1.049M      0.06%


In [84]:
trainer.fit(classfier_lightning_model, train_dataloaders=train_dataloader, val_dataloaders=test_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                   | Params
-----------------------------------------------------
0 | model     | GPTEmbeddingClassifier | 918 K 
1 | loss_func | BCEWithLogitsLoss      | 0     
2 | train_acc | MulticlassAccuracy     | 0     
3 | val_acc   | MulticlassAccuracy     | 0     
4 | test_acc  | MulticlassAccuracy     | 0     
-----------------------------------------------------
918 K     Trainable params
0         Non-trainable params
918 K     Total params
3.675     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\fardin\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
c:\Users\fardin\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=200` reached.


In [88]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from torchmetrics.classification import ConfusionMatrix
def calculate_metrics(cl_model):
    cm = ConfusionMatrix(task="multiclass", num_classes=len(class_id))

    y_pred = []
    y_true = []

    cl_model = cl_model.eval()
    cl_model.to(device)
    for X, y in tqdm(test_dataloader):
        X = X.to(device)
        with torch.no_grad():
            y_p = cl_model(X)
            y_p = y_p.cpu()
        y_pred.append(y_p)
        y_true.append(y)
    y_pred = torch.cat(y_pred, dim=0)
    y_true = torch.cat(y_true, dim=0)
    y_pred2 = torch.argmax(y_pred, dim=1)
    y_true2 = torch.argmax(y_true, dim=1)
    print(f'classification report: \n {classification_report(y_true2, y_pred2, digits=4)}')
    print(f'confusion matrix:\n {cm(y_pred2, y_true2)}')
    print('================================')


In [89]:
trainer.checkpoint_callback.best_model_path

'logs/log2\\version_97\\checkpoints\\epoch=77-step=6552.ckpt'

In [94]:
classifier_torch_model = GPTEmbeddingClassifier(embedding_dim, num_classes).to(device)
best_model_path = rf'{trainer.checkpoint_callback.best_model_path}' 
# best_model_path = r'logs\log2\version_97\checkpoints\epoch=143-step=12096.ckpt'
lightning_model = ClassifierLightningModel.load_from_checkpoint(best_model_path, map_location=None, hparams_file=None, strict=True, model=classifier_torch_model, num_classes=len(class_id)).eval()

c:\Users\fardin\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightning\pytorch\utilities\parsing.py:198: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [95]:
# classfier_lightning_model.model = classfier_lightning_model.model.eval()
# classfier_lightning_model = classfier_lightning_model.eval()
calculate_metrics(lightning_model)

100%|██████████| 9/9 [00:00<00:00, 39.13it/s]


classification report: 
               precision    recall  f1-score   support

           0     0.9834    0.9851    0.9842      4627
           1     0.9849    0.9832    0.9841      4589

    accuracy                         0.9842      9216
   macro avg     0.9842    0.9842    0.9842      9216
weighted avg     0.9842    0.9842    0.9842      9216

confusion matrix:
 tensor([[4558,   69],
        [  77, 4512]])


In [96]:
classfier_lightning_model.model = classfier_lightning_model.model.eval()
classfier_lightning_model = classfier_lightning_model.eval()
calculate_metrics(classfier_lightning_model)

  0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 9/9 [00:00<00:00, 36.00it/s]


classification report: 
               precision    recall  f1-score   support

           0     0.9810    0.9840    0.9825      4627
           1     0.9838    0.9808    0.9823      4589

    accuracy                         0.9824      9216
   macro avg     0.9824    0.9824    0.9824      9216
weighted avg     0.9824    0.9824    0.9824      9216

confusion matrix:
 tensor([[4553,   74],
        [  88, 4501]])
